In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
%matplotlib inline

import sys, os, csv, datetime
import glob
from typing import Dict
parent_directory = os.path.split(os.getcwd())[0]
src_directory = os.path.join(parent_directory, 'src')
data_directory = os.path.join('/scratch/groups/gracegao', 'Google_GNSS_data')
ephemeris_data_directory = os.path.join(data_directory, 'ephemeris')
sys.path.insert(0, src_directory)

import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import xarray as xr

In [31]:
from utils import *

from gnss_lib.sim_gnss import expected_measures, precompute_prange_satpos
import gnss_lib.sim_gnss as sgnss
from gnss_lib.utils import datetime_to_tow 
from gnss_lib import coordinates as coord

In [32]:
from tqdm.notebook import tqdm

In [33]:
## HyperParams
data_config = {
    "old_root": os.path.join(data_directory, "train_new"),
    "new_root": os.path.join(data_directory, "android_small_rebatched_1")
    }

In [34]:
initialize_dirs(data_config["new_root"])
dir_paths = setup_dir(data_config["new_root"], ["gnss_data", "processed"])

In [42]:
debug_res_list = []
def debug_residuals(data, satXYZV, true_XYZb, prange_corr):
    delXYZ, true_range = sgnss._find_delxyz_range(satXYZV, true_XYZb[:3], len(satXYZV.index))
    return true_range + prange_corr + true_XYZb[3] - data['prange'].to_numpy()

list1 = sorted_list(data_config["old_root"], ext="/*/device_gnss.csv")
list1_gt = sorted_list(data_config["old_root"], ext="/*/ground_truth.csv")

100%|██████████| 119/119 [00:00<00:00, 434397.02it/s]


In [104]:
# Carrier smoothing of pseudarange
def carrier_smoothing(gnss_df):
    """
    Args:
        df : DataFrame from device_gnss.csv
    Returns:
        df: DataFrame with carrier-smoothing pseudorange 'pr_smooth'
    """
    carr_th = 1.6 # carrier phase jump threshold [m] 2->1.5 (best)->1.0
    pr_th =  20.0 # pseudorange jump threshold [m] 20->15

    prsmooth = np.full_like(gnss_df['RawPseudorangeMeters'], np.nan)
    # Loop for each signal
    for (i, (svid_sigtype, df)) in enumerate((gnss_df.groupby(['Svid', 'SignalType']))):
        df = df.replace(
            {'AccumulatedDeltaRangeMeters': {0: np.nan}})  # 0 to NaN

        # Compare time difference between pseudorange/carrier with Doppler
        drng1 = df['AccumulatedDeltaRangeMeters'].diff() - df['PseudorangeRateMetersPerSecond']
        drng2 = df['RawPseudorangeMeters'].diff() - df['PseudorangeRateMetersPerSecond']

        # Check cycle-slip
        slip1 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**1) != 0  # reset flag
        slip2 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**2) != 0  # cycle-slip flag
        slip3 = np.fabs(drng1.to_numpy()) > carr_th # Carrier phase jump
        slip4 = np.fabs(drng2.to_numpy()) > pr_th # Pseudorange jump

        idx_slip = slip1 | slip2 | slip3 | slip4
        idx_slip[0] = True

        # groups with continuous carrier phase tracking
        df['group_slip'] = np.cumsum(idx_slip)

        # Psudorange - carrier phase
        df['dpc'] = df['RawPseudorangeMeters'] - df['AccumulatedDeltaRangeMeters']

        # Absolute distance bias of carrier phase
        meandpc = df.groupby('group_slip')['dpc'].mean()
        df = df.merge(meandpc, on='group_slip', suffixes=('', '_Mean'))

        # Index of original gnss_df
        idx = (gnss_df['Svid'] == svid_sigtype[0]) & (
            gnss_df['SignalType'] == svid_sigtype[1])

        # Carrier phase + bias
        prsmooth[idx] = df['AccumulatedDeltaRangeMeters'] + df['dpc_Mean']

    # If carrier smoothing is not possible, use original pseudorange
    idx_nan = np.isnan(prsmooth)
    prsmooth[idx_nan] = gnss_df['RawPseudorangeMeters'][idx_nan]
    gnss_df['pr_smooth'] = prsmooth

    return gnss_df

In [120]:
sigtype_dict = {
    'GPS_L1': 1,
    'GPS_L5': 2,
    'GLO_G1': 3,
    'QZS_J1': 4,
    'QZS_J5': 5,
    'BDS_B1I': 6,
    'GAL_E1': 7,
    'GAL_E5A': 8,
}

sigtype2idx = lambda x: np.array([sigtype_dict[_x] for _x in x])

In [121]:
# for j in tqdm(range(len(list1)), desc='files'):
j = 0
filepath = list1[j]
fulldata = pd.read_csv(filepath)
fulldata_gt = pd.read_csv(list1_gt[j])
outname = filepath.split('/')[-1][:-4]
list2 = fulldata['utcTimeMillis'].unique()
for i in tqdm(range(len(list2))):
    timestep = list2[i]
    data = fulldata.loc[fulldata['utcTimeMillis']==timestep]
    data_gt = fulldata_gt.loc[fulldata_gt['UnixTimeMillis']==timestep]

    # Calculate corrections
    gpsweek, tow = datetime_to_tow(datetime.datetime.fromtimestamp(timestep/1e3))
    ephem = data.set_index('Svid')

    _data0 = data.iloc[0]

    wls_lla = coord.ecef2geodetic(list(_data0[['WlsPositionXEcefMeters', 'WlsPositionYEcefMeters', 'WlsPositionZEcefMeters']].to_numpy()))
    gt_lla = data_gt[['LatitudeDegrees', 'LongitudeDegrees', 'AltitudeMeters']].to_numpy()
    fused_lla = np.array([gt_lla[0, 0], gt_lla[0, 1], wls_lla[2]])
    fused_ecef = coord.geodetic2ecef(fused_lla)

    # Change to WLS or some other approximate initialization
    true_XYZb = np.array([fused_ecef[0], fused_ecef[1], fused_ecef[2], 0.0, 0.0, 0.0, 0.0, 0.0])

    # Generate expected measures and satellite positions/velocities
    satXYZ = data[['SvPositionXEcefMeters', 'SvPositionYEcefMeters', 'SvPositionZEcefMeters']].to_numpy()
    expected_prange = np.linalg.norm(satXYZ - true_XYZb[None, :3], axis=-1)
    data = carrier_smoothing(data)

    prange_corr = (data['SvClockBiasMeters'] - data['IsrbMeters'] -
                  data['IonosphericDelayMeters'] - data['TroposphericDelayMeters']).to_numpy()

    sim_data = {
        "timestamp": timestep, 
        "gt_x": true_XYZb[0],
        "gt_y": true_XYZb[1],
        "gt_z": true_XYZb[2],
        "gt_b": true_XYZb[3],
        "gt_vx": true_XYZb[4],
        "gt_vy": true_XYZb[5],
        "gt_vz": true_XYZb[6],
        "gt_vb": true_XYZb[7],
        "prange": data['pr_smooth'].to_numpy(), 
        "prange_corr": prange_corr, 
        "doppler": data['PseudorangeRateMetersPerSecond'].to_numpy(), 
        "multipath": data['MultipathIndicator'].to_numpy(),
        "sigtype": sigtype2idx(data['SignalType'].to_numpy()),
        "cn0": data['Cn0DbHz'].to_numpy(),
        "sv_x": data['SvPositionXEcefMeters'].to_numpy(),
        "sv_y": data['SvPositionYEcefMeters'].to_numpy(),
        "sv_z": data['SvPositionZEcefMeters'].to_numpy(),
        "sv_vx": data['SvVelocityXEcefMetersPerSecond'].to_numpy(),
        "sv_vy": data['SvVelocityYEcefMetersPerSecond'].to_numpy(),
        "sv_vz": data['SvVelocityZEcefMetersPerSecond'].to_numpy(),
               }

    new_data = pd.DataFrame(data=sim_data)

    new_data.to_hdf(os.path.join(dir_paths[0], outname+'_'+str(i)+'.h5'), key='new_data')

  0%|          | 0/3362 [00:00<?, ?it/s]

<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

...

<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df['pr_smooth'] = prsmooth
<ipython-input-104-fcf25cd27175>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [ ]:
# # PLOT RESIDUALS
# for i, res_l in enumerate(debug_res_list):
#     plt.scatter([i]*len(res_l), res_l)

# # PLOT TRAJECTORY
# i = 1

# plt.figure()
# plt.plot(traj_data[i, :, 0])

# plt.figure()
# plt.plot(traj_data[i, :, 1])

# plt.figure()
# plt.plot(traj_data[i, :, 2])

# plt.figure()
# plt.plot(traj_data[i, :, 3])

In [122]:
tmp = pd.read_hdf('/scratch/groups/gracegao/Google_GNSS_data/android_small_rebatched_1/gnss_data/device_gnss_1.h5', 'new_data')

In [123]:
tmp

,timestamp,gt_x,gt_y,gt_z,gt_b,gt_vx,gt_vy,gt_vz,gt_vb,prange,...,doppler,multipath,sigtype,cn0,sv_x,sv_y,sv_z,sv_vx,sv_vy,sv_vz
0,1589573680445,-2.693912e+06,-4.297452e+06,3.854198e+06,0.0,0.0,0.0,0.0,0.0,2.084635e+07,...,-175.322556,0,1,40.1,-5.062681e+06,-2.369242e+07,1.088275e+07,847.270252,1097.387907,2801.347838
1,1589573680445,-2.693912e+06,-4.297452e+06,3.854198e+06,0.0,0.0,0.0,0.0,0.0,2.239090e+07,...,-491.709320,0,1,31.3,-1.729803e+07,1.642844e+06,1.980298e+07,1362.348941,-2178.209272,1370.002927
2,1589573680445,-2.693912e+06,-4.297452e+06,3.854198e+06,0.0,0.0,0.0,0.0,0.0,2.273144e+07,...,474.183289,0,1,25.3,7.919596e+06,-1.388941e+07,2.162183e+07,2632.438508,222.546494,-842.137951
3,1589573680445,-2.693912e+06,-4.297452e+06,3.854198e+06,0.0,0.0,0.0,0.0,0.0,2.087589e+07,...,360.951172,0,1,30.3,-8.455094e+05,-1.472873e+07,2.178728e+07,2817.585475,293.257352,301.586709
4,1589573680445,-2.693912e+06,-4.297452e+06,3.854198e+06,0.0,0.0,0.0,0.0,0.0,2.106533e+07,...,-47.307243,0,1,39.3,-1.954838e+07,-9.151281e+06,1.551392e+07,-897.386276,-1799.517520,-2128.372068
5,1589573680445,-2.693912e+06,-4.297452e+06,3.854198e+06,0.0,0.0,0.0,0.0,0.0,2.505150e+07,...,-597.636292,0,1,16.3,-1.855122e+07,1.209514e+07,1.422095e+07,747.823315,-1714.719065,2419.665601
6,1589573680445,-2.693912e+06,-4.297452e+06,3.854198e+06,0.0,0.0,0.0,0.0,0.0,2.370524e+07,...,405.060364,0,1,35.9,1.166709e+07,-2.328928e+07,6.145896e+06,571.357614,-451.800264,-3077.311646
7,1589573680445,-2.693912e+06,-4.297452e+06,3.854198e+06,0.0,0.0,0.0,0.0,0.0,1.976310e+07,...,414.793488,0,3,31.4,-3.623732e+06,-1.339991e+07,2.136985e+07,2459.126470,1603.217620,1433.744381
8,1589573680445,-2.693912e+06,-4.297452e+06,3.854198e+06,0.0,0.0,0.0,0.0,0.0,2.059145e+07,...,-556.043457,0,3,24.3,-1.081862e+07,-3.859187e+06,2.275605e+07,-33.424563,-3121.037220,-545.038538
9,1589573680445,-2.693912e+06,-4.297452e+06,3.854198e+06,0.0,0.0,0.0,0.0,0.0,1.935394e+07,...,141.793060,0,3,38.6,-6.280729e+06,-2.069199e+07,1.357049e+07,-124.443815,-1853.994642,-2887.583836
